<a href="https://colab.research.google.com/github/TanisiCurtin/Programming-Assignment-2/blob/main/Business_Report_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Programming Assignment 2
Australian Energy Dataset Analysis
Business Reports

Lets Setup an Environment:
A New colab notebook alog with new repository at my GitHub Account is created.

Lets connect it to Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Following libraries are imported:

In [4]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Connecting to the SQLite Database

In [8]:
db_path='/content/drive/My Drive/AUS_energy_sqlite3.db'
conn = sqlite3.connect(db_path)
#Verifying connection and checking the database schema
cursor=conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('AUS_energy',)]


Data Extraction and Manipulation
Example SQL query to load data into a DataFrame

In [ ]:
##lets display the Table AUS_energy
table_name='AUS_energy'
query= f"SELECT * FROM {table_name}"
with pd.option_context('display.max_rows',None): ##to display all rows
    df=pd.read_sql_query(query,conn)
print(df)